Author: Ewa Komkowska, 136737

# Algorithm 

### Main structure

In [8]:
import numpy as np
from statistics import median

class Node(object):
  def __init__(self):
    self.left = None 
    self.right = None 
    self.value = None
    self.predict_value = None
    self.predict_index = None
		
  def perform_split(self, data, function_value):
    self.value = function_value
    data_to_deviation = np.zeros((len(data[0]), len(data)), dtype=float)

    for i, elem_list in enumerate(data):
      for index, attribute_value in enumerate(elem_list):
        data_to_deviation[index][i] = attribute_value

    deviation = [np.std(data_to_deviation[index]) for index in range(len(data_to_deviation))]

    max_deviation = max(deviation[:-1])
    deviation_index = deviation.index(max_deviation)
    median_value = median(data_to_deviation[deviation_index])
    
    d1 = [elem for elem in data if not elem[deviation_index] < median_value]
    d2 = [elem for elem in data if elem[deviation_index] < median_value]

    if len(d1) > 5 or len(d2) > 5:
      left_average = sum([elem[-1] for elem in d1]) / len(d1) if len(d1) else 0
      right_average = sum([elem[-1] for elem in d2]) / len(d2) if len(d2) else 0

      provide_value_left = sum([(left_average - elem[-1]) ** 2 for elem in data if not elem[deviation_index] < median_value])
      provide_value_right = sum([(right_average - elem[-1]) ** 2 for elem in data if elem[deviation_index] < median_value])

      if len(d1) > 5 and provide_value_left < function_value and left_average != 0: 
        self.left = Node()
        self.value = left_average
        self.predict_value = median_value
        self.predict_index = deviation_index
        self.left.perform_split(d1, provide_value_left)
        
        if self.left.perform_split(d1, provide_value_left) is None:
            self.value = function_value
            self.left = None
      
      if len(d2) > 5 and provide_value_right < function_value and right_average != 0: 
        self.right = Node()
        self.value = right_average
        self.predict_value = median_value
        self.predict_index = deviation_index
        self.right.perform_split(d2, provide_value_right)
        
        if self.right.perform_split(d2, provide_value_right) is None:
            self.value = function_value
            self.right = None
    else:
      self.value = sum([elem[-1] for elem in d1]) + sum([elem[-1] for elem in d2]) / (len(d1) + len(d2))

    return self.value

  def predict(self, example):
    if self.predict_index is not None and self.predict_value is not None:
      if example[self.predict_index] < self.predict_value:
        if self.right is not None:
          return self.right.predict(example)
        else:
          return self.value
      elif self.left is not None:
          return self.left.predict(example)

    return self.value

### Read data and run algorithm

In [ ]:
for i in range(11,12):
  id = str(i)
  data = []
  y = [line.strip() for line in open('sample_data/' + id + '-Y.csv')]
  for i, line in enumerate(open('sample_data/' + id + '-X.csv')):
    if i == 0:
      continue
    x = [float(j) for j in line.strip().split(',')]
    nAttr = len(x)
    x.append(float(y[i]))
    data.append(x)
  print('Data load complete!')
  tree_root = Node()
  tree_root.perform_split(data, 99999999)
  print('Training complete!')

  with open('final/' + id + '.csv', 'w') as f:
    for i, line in enumerate(open('sample_data/' + id + '-test.csv')):
      if i == 0:
        continue
      x = [float(j) for j in line.strip().split(',')]
      y = tree_root.predict(x)
      f.write(str(y))
      f.write('\n')

Data load complete!
